In [ ]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END

from langgraph.graph.message import add_messages



In [ ]:
class State(TypedDict):
    messages : Annotated[list, add_messages]



In [11]:
from langchain_groq import ChatGroq
from langchain.chat_models import init_chat_model

llm= ChatGroq(model="llama3-8b-8192")

In [12]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001F26F3F0D70>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F26F3F1940>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
def chatbot(state:State):
    return {"messages":[llm.invoke(state["messages"])]}
    

In [ ]:
graph_builder = StateGraph(State)
graph_builder.add_node("basicChatbot",chatbot)
graph_builder.add_edge(START,"basicChatbot")
graph_builder.add_edge("basicChatbot",END)
graph=graph_builder.compile()

In [ ]:
from IPython.display import Image,display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass


In [ ]:
response=graph.invoke({"messages":"Hi"})

In [ ]:
response["messages"][-1].content

In [10]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
graph.invoke({"messages":"Provide me the recent AI news"})

In [9]:
from langchain_tavily import TavilySearch

tool = TavilySearch(max_results=2)
tool.invoke("What is langgraph?")

{'query': 'What is langgraph?',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.datacamp.com/tutorial/langgraph-tutorial',
   'title': 'LangGraph Tutorial: What Is LangGraph and How to Use It?',
   'content': 'LangGraph is a library within the LangChain ecosystem that provides a framework for defining, coordinating, and executing multiple LLM agents (or chains) in a structured and efficient manner. By managing the flow of data and the sequence of operations, LangGraph allows developers to focus on the high-level logic of their applications rather than the intricacies of agent coordination. Whether you need a chatbot that can handle various types of user requests or a multi-agent system that performs complex tasks, LangGraph provides the tools to build exactly what you need. LangGraph significantly simplifies the development of complex LLM applications by providing a structured framework for managing state and coordinating agent interactio

In [6]:
def multiply(a:int,b:int)->int:
    """Multiply a and b
    
    Args:
        a (int): first int
        b (int): second int

    Returns:
        int : output int
    """
    return a*b

In [5]:
from langgraph.types import Command, interrupt
from langchain_tavily import TavilySearch
from langchain_core.tools import tool

@tool
def human_assistance(query : str) -> str:
    """ Request assistance from human """
    human_response = interrupt({"query": query})
    return human_response["data"]

tools=[tool,human_assistance]


In [7]:
tools

[<function langchain_core.tools.convert.tool(name_or_callable: Union[str, Callable, NoneType] = None, runnable: Optional[langchain_core.runnables.base.Runnable] = None, *args: Any, description: Optional[str] = None, return_direct: bool = False, args_schema: Union[type[pydantic.main.BaseModel], dict[str, Any], NoneType] = None, infer_schema: bool = True, response_format: Literal['content', 'content_and_artifact'] = 'content', parse_docstring: bool = False, error_on_invalid_docstring: bool = True) -> Union[langchain_core.tools.base.BaseTool, Callable[[Union[Callable, langchain_core.runnables.base.Runnable]], langchain_core.tools.base.BaseTool]]>,
 StructuredTool(name='human_assistance', description='Request assistance from human', args_schema=<class 'langchain_core.utils.pydantic.human_assistance'>, func=<function human_assistance at 0x000001F26F10CC20>)]

In [13]:
llm_with_tools=llm.bind_tools(tools)

ValueError: Arg Precedence for the tool description value is as follows in docstring not found in function signature.

In [14]:
llm_with_tools

NameError: name 'llm_with_tools' is not defined

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

def tool_calling_llm(state: State) :
    return {"messages":[llm_with_tools.invoke(state["messages"])]} 


builder= StateGraph(State)
builder.add_node("tool_calling_llm",tool_calling_llm)
builder.add_node("tools",ToolNode(tools))

builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges(
    "tool_calling_llm",
    #If the latest message from assistant is a tool call, tools_condition routes to tool
    #If the latest message from assistant is not a tool call, tools_condition routes to END
    tools_condition
)
builder.add_edge("tools",END)

graph=builder.compile()

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass

In [ ]:
graph.invoke({"messages":"what is the latest AI News"})

In [ ]:
response = graph.invoke({"messages":"what is the latest AI News"})

In [ ]:
response["messages"][-1].content

In [ ]:
for m in response["messages"]:
    m.pretty_print()

In [ ]:
response = graph.invoke({"messages":"hi, how are u? what is the latest ai news and then 2 multiply by 3 then multiply by 10"})
for m in response["messages"]:
    m.pretty_print()

In [4]:
### ReAct Architecture with memory

from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
from langgraph.checkpoint.memory import MemorySaver

memory=MemorySaver()

def tool_calling_llm(state: State) :
    return {"messages":[llm_with_tools.invoke(state["messages"])]} 
builder= StateGraph(State)
builder.add_node("tool_calling_llm",tool_calling_llm)
builder.add_node("tools",ToolNode(tools))

builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges(
    "tool_calling_llm",
    #If the latest message from assistant is a tool call, tools_condition routes to tool
    #If the latest message from assistant is not a tool call, tools_condition routes to END
    tools_condition
)
builder.add_edge("tools","tool_calling_llm")

graph=builder.compile(checkpointer=memory)

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass

NameError: name 'State' is not defined

In [ ]:
config={"configurable":{"thread_id":"1"}}
response=graph.invoke({"messages":"Hi, do you remember my name?"},config=config)
print(response["messages"][-1].content)

In [ ]:
response = graph.invoke({"messages":"hi, how are u? what is the latest ai news and then 2 multiply by 3 then multiply by 10"})
for m in response["messages"]:
    m.pretty_print()

In [ ]:
def superbot(state:State):
    return {"messages":[llm.invoke(state["messages"])]}

In [ ]:
builder = StateGraph(State)
builder.add_node("SuperBot", superbot)
builder.add_edge(START,"SuperBot" )
builder.add_edge("SuperBot",END )
graph_builder=builder.compile(checkpointer=memory)

try:
    display(Image(graph_builder.get_graph().draw_mermaid_png()))
except Exception:
    pass

In [ ]:
config={"configurable":{"thread_id":"1"}}
response=graph_builder.invoke({"messages":"Hi, My name is Soumya, I like to dance?"},config=config)
print(response["messages"][-1].content)

In [ ]:
config={"configurable":{"thread_id":"2"}}
for chunk in graph_builder.stream({"messages":"Hi, My name is Soumya, I like to dance?"},config=config,stream_mode='updates'):
    print(chunk)

In [ ]:
config={"configurable":{"thread_id":"4"}}
for chunk in graph_builder.stream({"messages":"Hi, My name is Soumya, I like to dance"},config=config,stream_mode='values'):
    print(chunk)

In [ ]:
for chunk in graph_builder.stream({"messages":"I also like to sing"},config=config,stream_mode='values'):
    print(chunk)

In [15]:
### ReAct Architecture with memory and human feedback
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain_tavily import TavilySearch
from langchain_core.tools import tool
from langgraph.types import Command, interrupt
from langchain_groq import ChatGroq
from langchain.chat_models import init_chat_model

llm= ChatGroq(model="llama3-8b-8192")

class State(TypedDict):
    messages : Annotated[list,add_messages]

memory=MemorySaver()

def tool_calling_llm(state: State) :
    return {"messages":[llm_with_tools.invoke(state["messages"])]} 

new_builder= StateGraph(State)

@tool
def human_assistance(query : str) -> str:
    """ Request assistance from human """
    human_response = interrupt({"query": query})
    return human_response["data"]

tool = TavilySearch(max_results=2)
tools = [tool,human_assistance]
llm_with_new_tools=llm.bind_tools(tools)

def chat_bot(state: State) :
    message = llm_with_new_tools.invoke(state["messages"])
    return {"messages" : [message]}

new_builder.add_node("chat_bot",chat_bot)
new_builder.add_node("tools",ToolNode(tools))

new_builder.add_edge(START, "chat_bot")
new_builder.add_conditional_edges(
    "chat_bot",
    #If the latest message from assistant is a tool call, tools_condition routes to new_tools
    #If the latest message from assistant is not a tool call, tools_condition routes to END
    tools_condition
)
new_builder.add_edge("tools","chat_bot")

graph_builder_new=new_builder.compile(checkpointer=memory)

try:
    display(Image(graph_builder_new.get_graph().draw_mermaid_png()))
except Exception:
    pass

In [16]:
user_input = "I need some expert guidance for building an AI Agent. Could you request assistance for me?"
config={"configurable":{"thread_id":"1"}}
for chunk in graph_builder_new.stream({"messages":user_input},config=config,stream_mode='values'):
    print(chunk)




{'messages': [HumanMessage(content='I need some expert guidance for building an AI Agent. Could you request assistance for me?', additional_kwargs={}, response_metadata={}, id='12f4d2b8-650b-4b91-9612-208e66c94967')]}
{'messages': [HumanMessage(content='I need some expert guidance for building an AI Agent. Could you request assistance for me?', additional_kwargs={}, response_metadata={}, id='12f4d2b8-650b-4b91-9612-208e66c94967'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'j0dnypwxr', 'function': {'arguments': '{"query":"I need some expert guidance for building an AI Agent. Could you request assistance for me?"}', 'name': 'human_assistance'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 93, 'prompt_tokens': 2885, 'total_tokens': 2978, 'completion_time': 0.154827305, 'prompt_time': 0.380573049, 'queue_time': 0.7159117460000001, 'total_time': 0.535400354}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_5b339000ab', 'servic

In [ ]:
human_response = (
    "We are here to help. Please checkout LangGraph"
    "its reliable and extendable"
)

human_command = Command(resume={"data": human_response})

for chunk in graph_builder_new.stream(human_command,config=config,stream_mode="values"):
    print(chunk)


In [18]:
from langgraph.types import Command, interrupt

# Your existing setup code...
# ... llm, new_builder, etc.
# graph_builder_new = new_builder.compile(checkpointer=memory)

# Step 1: Run the graph until it encounters the interrupt
user_input = "I need some expert guidance for building an AI Agent. Could you request assistance for me?"
config = {"configurable": {"thread_id": "1"}}

print("--- Running the graph (should interrupt at the human_assistance tool) ---")
try:
    for chunk in graph_builder_new.stream({"messages": user_input}, config=config, stream_mode='values'):
        # This will print the messages as they are generated by the graph
        # and the stream will stop when it yields the Command object.
        print(f"Graph Output: {chunk}")
except Exception as e:
    print(f"An error occurred during the first stream call: {e}")
    # If an error happens here, the second loop will not be reached.

print("--- The graph is now interrupted. It is waiting for the human command. ---")

# Step 2: Provide the human response to resume the graph
human_response = (
    "We are here to help. Please checkout LangGraph"
    "its reliable and extendable"
)
human_command = Command(resume={"data": human_response})

print("--- Sending the human command to resume the graph ---")
events = graph_builder_new.stream(human_command,config,stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()


--- Running the graph (should interrupt at the human_assistance tool) ---
Graph Output: {'messages': [HumanMessage(content='I need some expert guidance for building an AI Agent. Could you request assistance for me?', additional_kwargs={}, response_metadata={}, id='12f4d2b8-650b-4b91-9612-208e66c94967'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'j0dnypwxr', 'function': {'arguments': '{"query":"I need some expert guidance for building an AI Agent. Could you request assistance for me?"}', 'name': 'human_assistance'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 93, 'prompt_tokens': 2885, 'total_tokens': 2978, 'completion_time': 0.154827305, 'prompt_time': 0.380573049, 'queue_time': 0.7159117460000001, 'total_time': 0.535400354}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_5b339000ab', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--064be175-b37d-4e13-baf7-335b90dc7220-0', tool_cal

In [2]:
# Step A: List available symbols
import langgraph
print(dir(langgraph))



['__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '_typing', 'cache', 'channels', 'checkpoint', 'config', 'constants', 'errors', 'graph', 'managed', 'prebuilt', 'pregel', 'store', 'types', 'typing', 'utils', 'warnings']
